In [1]:
from tensorflow import keras
eyesModel = keras.models.load_model('eyesOpenClose.h5')
yawnModel = keras.models.load_model('yawn.h5')

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import time

# Stop button
# ================
stopButton = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
display_handle=display(None, display_id=True)

detector = cv2.FaceDetectorYN.create(
        'face_detection_yunet_2022mar.onnx',
        "",
        (320, 320),
        0.9, # score threshold
        0.3, # nms threshold
        5000 # top k
    )

preds = []
start_time = time.perf_counter()
closed_imgs = []
open_imgs = []
def view(button):
    while True:
        _, frame = cap.read()
        detector.setInputSize((frame.shape[1], frame.shape[0]))
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        
        faces = detector.detect(frame)[1] # face_cascade.detectMultiScale(frame, 1.3, 5)
        eyes = [] # eye_cascade.detectMultiScale(frame, scaleFactor = 1.05, minNeighbors = 1)
        
        face_boxes = []
        if faces is not None:
            for face in faces:
                x,y,w,h = face[:4].astype('int')
                face_boxes.append([x,y,x+w,y+h])
                
            lx,ly,la,lb = sorted([[x,y,a,b,(a-x)*(b-y)] for x,y,a,b in face_boxes], key=lambda x: x[4])[-1][:-1]
            face_array = frame[max(0,ly):min(lb,frame.shape[0]), max(0,lx):min(la,frame.shape[1])]
#             if face_array.shape[0] > 0 and face_array.shape[1]:
            face_array = cv2.resize(face_array, (100,100)).reshape(-1, 100, 100, 3)
#             _, arr = cv2.imencode('.jpeg', face_array[0])
#             display(Image(data=arr.tobytes()))
#             face_array = face_array / 255
            yawn_pred = yawnModel.predict(face_array/255, verbose=0).item()
            yawn_text = "Mouth: Open" if yawn_pred < 0.2 else "Mouth: Closed"
            cv2.putText(frame, yawn_text, (frame.shape[1]-250,frame.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            eyes_pred = eyesModel.predict(face_array/255, verbose=0).item()
            eyes_text = "Eyes: Closed" if eyes_pred < 0.50 else "Eyes: Open"
            cv2.putText(frame, eyes_text, (25,frame.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


            frame = cv2.rectangle(frame, (lx, ly), (la, lb), (0, 255, 0), 2)
            # get largest face   
            

#         for (x, y, w, h) in eyes:
#             if any([box[0] <= x and box[1] <= y and x+w <= box[2] and y+h <= box[3] for box in face_boxes]):
#                 frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
#         display(Image(data=frame.tobytes()))
#         time.sleep(0.25)
            
            
def closer(b):
    print(time.perf_counter()-start_time)
    cap.release()
    display_handle.update(None)
            
stopButton.on_click(closer)
            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

None

Button(button_style='danger', description='Stop', icon='square', style=ButtonStyle(), tooltip='Description')

52.6388269


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\fgpac\anaconda3\envs\bhacks\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\fgpac\anaconda3\envs\bhacks\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\fgpac\AppData\Local\Temp\ipykernel_11596\849121716.py", line 43, in view
AttributeError: 'NoneType' object has no attribute 'shape'
